<a href="https://colab.research.google.com/github/seoes/odyssey-ai/blob/seo/supervision_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output, display, Image
import io

In [2]:
%pip install ultralytics
!pip install supervision
from ultralytics import YOLO
import supervision as sv

model = YOLO("yolov8n.pt")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 1.0 MB/s eta 0:00:00


100%|██████████| 6.23M/6.23M [00:00<00:00, 57.6MB/s]


In [12]:
import os
HOME = os.getcwd()
%cd {HOME}
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=10pVQ9rSIJOOV69J8zkp9svozXfaa7-4w' -O sample-mall.mp4
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vVrEVMxucHgqGd7vAa501ASojbeGPhIr' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vVrEVMxucHgqGd7vAa501ASojbeGPhIr" -O market-square.mp4 && rm -rf /tmp/cookies.txt
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=10pVQ9rSIJOOV69J8zkp9svozXfaa7-4w' -O market-sqaure.mp4

# SAMPLE_VIDEO_PATH = f"{HOME}/sample-mall.mp4"
SAMPLE_VIDEO_PATH = f"{HOME}/market-square.mp4"

/content
--2023-11-01 06:01:24--  https://drive.google.com/uc?export=download&id=10pVQ9rSIJOOV69J8zkp9svozXfaa7-4w
Resolving drive.google.com (drive.google.com)... 108.177.12.113, 108.177.12.102, 108.177.12.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.12.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0ft0crkbb2mf6v2do1tn554f3qjvuude/1698818475000/09162896557746415517/*/10pVQ9rSIJOOV69J8zkp9svozXfaa7-4w?e=download&uuid=5b4173fc-3d6e-4cbd-ac74-d1d0e85c0f9d [following]
--2023-11-01 06:01:25--  https://doc-0o-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0ft0crkbb2mf6v2do1tn554f3qjvuude/1698818475000/09162896557746415517/*/10pVQ9rSIJOOV69J8zkp9svozXfaa7-4w?e=download&uuid=5b4173fc-3d6e-4cbd-ac74-d1d0e85c0f9d
Resolving doc-0o-0g-docs.googleusercontent.com (doc-0o-0g-docs.googleusercontent.com)... 142.250.97.1

In [9]:
box_annotator = sv.BoundingBoxAnnotator(thickness=6)

In [10]:
tracker = sv.ByteTrack()

In [14]:
def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[(detections.class_id == 0) & (detections.confidence > 0.2)]
    frame = box_annotator.annotate(frame.copy(), detections=detections)
    labels = [
      f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
      for _, _, confidence, class_id, tracker_id
      in detections
    ]
    return frame


sv.process_video(
    source_path=SAMPLE_VIDEO_PATH,
    target_path=f"{HOME}/sample-market-result.mp4",
    callback=callback
)


0: 640x384 16 persons, 2 umbrellas, 195.9ms
Speed: 6.9ms preprocess, 195.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 persons, 1 umbrella, 152.5ms
Speed: 3.9ms preprocess, 152.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 persons, 1 umbrella, 163.1ms
Speed: 4.0ms preprocess, 163.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 13 persons, 2 umbrellas, 171.7ms
Speed: 3.6ms preprocess, 171.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 12 persons, 2 umbrellas, 152.1ms
Speed: 6.8ms preprocess, 152.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 15 persons, 2 umbrellas, 149.5ms
Speed: 3.8ms preprocess, 149.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 persons, 2 umbrellas, 173.7ms
Speed: 4.0ms preprocess, 173.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

